In [1]:
# from IPython import display
import grequests
import requests
from lxml import html
import pandas as pd
import re
from configparser import ConfigParser

config = ConfigParser()
config.read('config.ini')
email = config['login']['email']
password = config['login']['password']

pd.set_option('display.max_rows', 100)

df = pd.DataFrame()

url = 'https://profitus.lt/users/login'

resp = ''

login_data = {}

def get_data():
  urls = []
  with requests.session() as client:
    client.get(url)
    csrftoken = client.cookies['csrfToken']
    login_data = dict(email=email, password=password, _csrfToken=csrftoken, next='/')
    r = client.post(url, data=login_data, headers=dict(Referer=url))
    resp = client.get('https://www.profitus.lt/secondary-market')
    tree = html.fromstring(resp.text)
    next_url = 'https://profitus.lt' + tree.xpath('//a[@class="page-link-p fw-700"]')[-1].values()[0]

    number_of_pages = int(re.search("\d+", next_url).group())

    for i in range(1, number_of_pages+1):
      urls.append(f'https://www.profitus.lt/secondary-market?page={i}')
      
    reqs = [grequests.get(link, headers=dict(Referer=url), session=client) for link in urls]
    resp = grequests.map(reqs)
    return resp

def parse_data(resp):
  global df
  for r in resp:
    print(r.url)
    df = pd.concat([df, pd.read_html(r.text)[0]])

  print('scraping is over!')


from time import time

current_time = time()

resp = get_data()
parse_data(resp)

print("Time taken: ", time() - current_time)

df = df.drop(['Unnamed: 0', 'Unnamed: 9'], axis=1)

df['Projekto pavadinimas'] = df['Projekto pavadinimas'].map(lambda x: x.lstrip('Projekto pavadinimas:&nbsp '))
df['Reitingas'] = df['Reitingas'].map(lambda x: x.lstrip('Reitingas:&nbsp '))
df['Likęs terminas'] = df['Likęs terminasi'].map(lambda x: x.lstrip('Likęs terminas:&nbsp ').rstrip('d.'))
df.drop('Likęs terminasi', axis=1, inplace=True)

# df['Likęs terminas'] = df['Likęs terminas'].map(lambda x: x.split('/')[0]).astype(int) + 1 # 1 month and 29 days is considered as 1 month, we're calculating worst example possible. only those will be worth to buy
df['Likęs terminas'] = df['Likęs terminas'].map(lambda x: x.split('/')[0]).astype(int) / 30 # 1 month and 29 days is considered as 1 month, we're calculating worst example possible. only those will be worth to buy


df['Likusi suma'] = df['Likusi suma'].map(lambda x: x.lstrip('Likusi suma:&nbsp'))

df['Likusi suma'] = df['Likusi suma'].str.replace('€', '')
df['Likusi suma'] = df['Likusi suma'].str.replace(',', '').astype(float)

df['Palūkanų norma'] = df['Palūkanų norma'].map(lambda x: x.lstrip('Palūkanų norma:&nbsp'))

df['Palūkanų norma'] = df['Palūkanų norma'].str.replace('%', '').astype(float)


df['Statusas'] = df['Statusas'].map(lambda x: x.lstrip('Statusas:&nbsp'))

# df['Statusas'] = df['Statusas'].str.replace('Aktyvus', '1')
# df['Statusas'] = df['Statusas'].str.replace('Vėluoja', '2').astype(int)



df['Likusi gautina sumai'] = df['Likusi gautina sumai'].map(lambda x: x.lstrip('Likusi gautina suma:&nbsp '))

df['Likusi gautina sumai'] = df['Likusi gautina sumai'].str.replace('€', '')
df['Likusi gautina sumai'] = df['Likusi gautina sumai'].str.replace(',', '').astype(float)

df['Pardavimo kaina'] = df['Pardavimo kaina'].map(lambda x: x.lstrip('Pardavimo kaina:&nbsp '))

df['Pardavimo kaina'] = df['Pardavimo kaina'].str.replace('€', '')
df['Pardavimo kaina'] = df['Pardavimo kaina'].str.replace(',', '').astype(float)

# df['Realios palūkanos'] = (df['Likusi gautina sumai'] - df['Pardavimo kaina']) * 100 / df['Pardavimo kaina'] / (df['Likęs terminas']/12)
import numpy as np
df[df['Likęs terminas'] < 1] = 1
df['Realios palūkanos'] = (np.power(df['Likusi gautina sumai'] / df['Pardavimo kaina'], 1/(df['Likęs terminas']/12)) - 1 ) * 100
df['diff'] = (df['Realios palūkanos'] - df['Palūkanų norma'])

print(df[df['diff'] > 0])
# print(df[(df['Palūkanų norma'] - df['Realios palūkanos']) < 0]) # basically we're trying to find projects that have largest interest rate, than
# print(df[df['Likęs terminas'] < 1].sort_values('Likęs terminas', ascending=True))
from IPython.display import display
# sent_mail = df[(df['Palūkanų norma'] - df['Realios palūkanos']) < 0]
sent_mail = df[df['diff'] > 0]
display(sent_mail)
df.sort_values('Realios palūkanos', ascending=False).head(100)





https://www.profitus.lt/secondary-market?page=1
https://www.profitus.lt/secondary-market?page=2
https://www.profitus.lt/secondary-market?page=3
https://www.profitus.lt/secondary-market?page=4
https://www.profitus.lt/secondary-market?page=5
https://www.profitus.lt/secondary-market?page=6
https://www.profitus.lt/secondary-market?page=7
https://www.profitus.lt/secondary-market?page=8
https://www.profitus.lt/secondary-market?page=9
https://www.profitus.lt/secondary-market?page=10
https://www.profitus.lt/secondary-market?page=11
https://www.profitus.lt/secondary-market?page=12
https://www.profitus.lt/secondary-market?page=13
https://www.profitus.lt/secondary-market?page=14
https://www.profitus.lt/secondary-market?page=15
https://www.profitus.lt/secondary-market?page=16
https://www.profitus.lt/secondary-market?page=17
scraping is over!
Time taken:  3.758554458618164
             Projekto pavadinimas Reitingas  Likusi suma  Palūkanų norma  \
5          #402 Tarandės namai II        B+       2

,Projekto pavadinimas,Reitingas,Likusi suma,Palūkanų norma,Statusas,Likusi gautina sumai,Pardavimo kaina,Likęs terminas,Realios palūkanos,diff
5,#402 Tarandės namai II,B+,200.00,6.5,Aktyvus,210.97,200.0,9.400000,7.054557,0.554557
0,#457 Link Ten V,A-,100.00,5.0,Aktyvus,103.42,100.0,6.366667,6.543478,1.543478
1,#424 Butai Kaune VI,A-,100.00,5.0,Aktyvus,104.81,100.0,11.500000,5.024300,0.024300
5,#396 Lapino namai IV,B-,860.00,8.0,Aktyvus,892.23,860.0,3.766667,12.435782,4.435782
2,#457 Link Ten V,A-,600.00,5.5,Aktyvus,622.60,600.0,6.366667,7.217613,1.717613
3,#492 Dvibutis Šatrininkų g. IV,B+,170.00,7.5,Aktyvus,179.05,170.0,7.666667,8.456892,0.956892
1,#306 Patalpos Laisvės al.,B+,200.00,6.5,Aktyvus,205.52,200.0,4.166667,8.156692,1.656692
2,#414 Namas Dusmenų g.,B-,200.00,7.5,Aktyvus,214.63,200.0,10.933333,8.056698,0.556698
4,#517 J7 Beržas III,C+,500.00,9.0,Aktyvus,522.44,511.0,2.733333,10.208345,1.208345
5,#390 Butai Kaune II,A-,2000.00,6.0,Aktyvus,2113.75,2010.0,9.600000,6.493219,0.493219


,Projekto pavadinimas,Reitingas,Likusi suma,Palūkanų norma,Statusas,Likusi gautina sumai,Pardavimo kaina,Likęs terminas,Realios palūkanos,diff
5,#396 Lapino namai IV,B-,860.00,8.0,Aktyvus,892.23,860.0,3.766667,12.435782,4.435782
0,#514 Kalno butai III,C,1000.00,9.0,Aktyvus,1089.51,1000.0,9.000000,12.109297,3.109297
4,#517 J7 Beržas III,C+,500.00,9.0,Aktyvus,522.44,511.0,2.733333,10.208345,1.208345
5,#467 LOFT206,B-,819.87,8.5,Aktyvus,904.83,819.0,12.933333,9.688114,1.188114
1,#477 Kalno butai II,C,500.00,8.5,Aktyvus,529.23,500.0,7.433333,9.605713,1.105713
3,#514 Kalno butai III,C,500.00,9.0,Aktyvus,544.75,511.0,9.000000,8.901815,-0.098185
2,#594 Aluonos sklypai,C-,100.00,14.0,Aktyvus,106.94,103.0,5.433333,8.644195,-5.355805
3,#526 Šatrininkų namai VII,B,100.00,9.0,Aktyvus,108.95,103.0,8.166667,8.602140,-0.397860
4,#526 Šatrininkų namai VII,B,100.00,9.0,Aktyvus,108.95,103.0,8.166667,8.602140,-0.397860
3,#492 Dvibutis Šatrininkų g. IV,B+,170.00,7.5,Aktyvus,179.05,170.0,7.666667,8.456892,0.956892


c:\Users\adoma\AppData\Local\Programs\Python\Python39\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [2]:
import smtplib, ssl
import io
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

port = 465
password = config['login']['from_password']

str_io = io.StringIO()
sent_mail.to_html(buf=str_io, classes='table table-striped')
html_str = str_io.getvalue()
context = ssl.create_default_context()

msg = MIMEMultipart('alternative')
msg['Subject'] = 'Profitus daily'
msg['From'] = "adomasval08@gmail.com"
msg['To'] = "adomasval04@gmail.com"
from datetime import datetime
text = f"Hi this is {datetime.now().strftime('%m-%d')} report"
part1 = MIMEText(text, 'plain')
part2 = MIMEText(html_str, 'html')

msg.attach(part1)
msg.attach(part2)

with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
  server.login("adomasval08@gmail.com", password)
  server.sendmail("adomasval08@gmail.com", "adomasval04@gmail.com", msg.as_string())

SMTPAuthenticationError: (535, b'5.7.8 Username and Password not accepted. Learn more at\n5.7.8  https://support.google.com/mail/?p=BadCredentials s13-20020a05651c200d00b0025a928f3d63sm1692531ljo.61 - gsmtp')

c:\Users\adoma\AppData\Local\Programs\Python\Python39\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [4]:
print(sent_mail)

          Projekto pavadinimas Reitingas  Likusi suma  Palūkanų norma  \
0            #347 Užupio butai         B        800.0             8.5   
3  #311 Butai Žvėryne „S41“ IV         A        100.0             5.0   

  Statusas  Likusi gautina sumai  Pardavimo kaina  Likęs terminas  \
0  Vėluoja                845.46            800.0        5.433333   
3  Aktyvus                102.15            101.0        2.666667   

   Realios palūkanos      diff  
0          12.982956  4.482956  
3           5.226830  0.226830  


C:\Users\adoma\AppData\Local\Programs\Python\Python39\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
